In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyrevolve
!pip install devito==4.2.3

In [ ]:
from devito import *

In [ ]:
import numpy as np
#NBVAL_IGNORE_OUTPUT
from examples.seismic import Model, plot_velocity

# Define a physical size
shape = (1001, 1001)  # Number of grid point (nx, nz)
spacing = (11., 11.)  # Grid spacing in m. The domain size is now
origin = (0., 0.)  # What is the location of the top left corner. This is necessary to define
# the absolute location of the source and receivers

# Define a velocity profile. The velocity is in km/s
v = np.empty(shape, dtype=np.float32)
v[:, :] = 2


# With the velocity and model size defined, we can create the seismic model that
# encapsulates this properties. We also define the size of the absorbing layer as 10 grid points
model = Model(vp=v, origin=origin, shape=shape, spacing=spacing,
              space_order=2, nbl=10, bcs="damp")

plot_velocity(model)

In [ ]:
from examples.seismic import TimeAxis

t0 = 0.  # Simulation starts a t=0
tn = 10000.  # Simulation last 10 second (10000 ms)
dt = model.critical_dt  # Time step from model grid spacing

time_range = TimeAxis(start=t0, stop=tn, step=dt)

In [ ]:
#NBVAL_IGNORE_OUTPUT
from examples.seismic import RickerSource

f0 = 0.020  # Source peak frequency is 10Hz (0.010 kHz)
src = RickerSource(name='src', grid=model.grid, f0=f0,
                   npoint=1, time_range=time_range)

# Set source coordinates
src.coordinates.data[0, 0] = 0.
src.coordinates.data[0, 1] = 0.  

# We can plot the time signature to see the wavelet via:
src.show()

In [ ]:
#NBVAL_IGNORE_OUTPUT
from examples.seismic import Receiver
import math

# Create symbol for 100 receivers
rec = Receiver(name='rec', grid=model.grid, npoint=100, time_range=time_range)

# Prescribe even spacing for receivers along the x-axis

circle_x=np.empty(100)
circle_y=np.empty(100)

for i in range(0,100):
    circle_x[i]= 10000. + 200. * math.cos((math.pi*i)/50)
    circle_y[i]= 10000. + 200. * math.sin((math.pi*i)/50)
    

rec.coordinates.data[:, 0] = circle_x[:]
rec.coordinates.data[:, 1] = circle_y[:]

# We can now show the source and receivers within our domain:
# Red dot: Source location
# Green dots: Receiver locations (every point)
plot_velocity(model, source=src.coordinates.data,
              receiver=rec.coordinates.data[::1, :])

In [ ]:
# In order to represent the wavefield u and the square slowness we need symbolic objects 
# corresponding to time-space-varying field (u, TimeFunction) and 
# space-varying field (m, Function)
from devito import TimeFunction

# Define the wavefield with the size of the model and the time dimension
u = TimeFunction(name="u", grid=model.grid, time_order=2, space_order=2)

# We can now write the PDE
pde = model.m * u.dt2 - u.laplace + model.damp * u.dt

# The PDE representation is as on paper
pde

In [ ]:
# This discrete PDE can be solved in a time-marching way updating u(t+dt) from the previous time step
# Devito as a shortcut for u(t+dt) which is u.forward. We can then rewrite the PDE as 
# a time marching updating equation known as a stencil using customized SymPy functions
from devito import Eq, solve

stencil = Eq(u.forward, solve(pde, u.forward))

In [ ]:
# Finally we define the source injection and receiver read function to generate the corresponding code
src_term = src.inject(field=u.forward, expr=src * dt**2 / model.m)

# Create interpolation expression for receivers
rec_term = rec.interpolate(expr=u.forward)

In [ ]:
#NBVAL_IGNORE_OUTPUT
from devito import Operator

op = Operator([stencil] + src_term + rec_term, subs=model.spacing_map)

In [ ]:
#NBVAL_IGNORE_OUTPUT
op(time=time_range.num-1, dt=model.critical_dt)

In [ ]:
#NBVAL_IGNORE_OUTPUT
from examples.seismic import plot_shotrecord

plot_shotrecord(rec.data, model, 000., 9000.)